#### Input

In [11]:
%load_ext autoreload
%matplotlib inline
%load_ext nb_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import helpplots as plothelp
import helpfunction as helper
import pickle

<IPython.core.display.Javascript object>

In [3]:
%autoreload

<IPython.core.display.Javascript object>

### Constants

In [4]:
pot_target = 1e20

<IPython.core.display.Javascript object>

In [5]:
input_dir = "./input/7Oct/run3/"
output_dir = "./output/run3/"

<IPython.core.display.Javascript object>

In [6]:
%%time
data = pickle.load(open(input_dir+"samples.pckl", 'rb'))
samples = list(data.keys())

CPU times: user 279 ms, sys: 374 ms, total: 653 ms
Wall time: 664 ms


<IPython.core.display.Javascript object>

### Plots